<a href="https://colab.research.google.com/github/postak/colazione-con-adk/blob/main/2025_09_Partners_ADK_Learning_session_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
Copyright 2025 Google LLC.
SPDX-License-Identifier: Apache-2.0
```

In [ ]:
#@title First Session
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 🚀 Welcome to the First Session of our Journey! 🚀

Welcome! In this notebook, we're going on a journey to build and command our own fleet of AI agents using the Google Agent Development Kit (ADK).

We'll cover three awesome levels of agent creation:
1.  **Your First Agent:** Learn the absolute basics of creating and running a single, powerful agent.
2.  **Agent with/without a Memory :** See how agents can remember your conversation within a 'session' to handle multi-step tasks.
3.  **Using ADK Tool in a shell environment:** See how the agent looks like in tool environment.

-------------
### ⚠️ Important Prerequisite: Setup Your Environment! ⚠️
-----------------------------------------------------------------------------
 Before you run this file, please make sure you have completed the initial
setup steps for Setting Up Your GCP Project & Gemini API Key.
👉 Follow the instructions [here](https://github.com/postak/colazione-con-adk/blob/main/Setting%20Up%20Your%20GCP%20Project%20%26%20Gemini%20API%20Key.pdf)

Once you've done that, you'll be all set to run this demo!
 -----------------------------------------------------------------------------


Let's get this adventure started!

## Part 0: Setup & Authentication 🔑

First things first, let's get all our tools ready. This step installs the necessary libraries and securely configures your Google API key so your agents can access the power of Gemini.

In [ ]:
!pip install google-adk google-generativeai -q

# --- Import all necessary libraries for our entire adventure ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ All libraries are ready to go!")

In [ ]:

# --- Securely Configure Your API Key ---

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Configure the generative AI library with the provided key
genai.configure(api_key=api_key)

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

---
## Lab 1: Your First Agent - Ask for the capital of a country 🧞

Meet your first creation! The `create_capital_agent` is a simple but powerful assistant. We're making it a little smarter by teaching it to understand **budget constraints**.

* **Agent**: The brain of the operation, defined by its instructions, tools, and the AI model it uses.
* **Session**: The conversation history. For this simple agent, it's just a container for a single request-response.
* **Runner**: The engine that connects the `Agent` and the `Session` to process your request and get a response.

### 1) Define an LLM agent function


In [ ]:
def create_capital_agent():
    """Create the Capital agent"""
    return Agent(
        name="capital_agent",
        model="gemini-2.0-flash",
        description="Answers user questions about the capital city of a given country.",
        instruction="""
        You are an agent that provides the capital city of a country.
        """
      )

capital_agent = create_capital_agent()
print(f"🧞 Agent '{capital_agent.name}' is created and ready for adventure!")

### 2) Define an helper function to run the agent (when not using adk tool)**

In [ ]:
# --- A Helper Function to Run Our Agents ---
# We'll use this function throughout the notebook to make running queries easy.

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {str(event)[:300]} \n...\n...\n<output truncated>\n\n")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"

### 3) Test the agent

In [ ]:
async def ask_for_nation_capital():
    # Create a new, single-use session for this query
    capital_session = await session_service.create_session(
        app_name=capital_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Which is the capital of Italy?"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(capital_agent, query, capital_session, my_user_id)

await ask_for_nation_capital()

### 4) Using Agent with a memory

In [ ]:
async def ask_for_nation_capital():
    # Create a new, single-use session for this query
    capital_session = await session_service.create_session(
        app_name=capital_agent.name,
        user_id=my_user_id
    )

    # Turn 1!
    query1 = "Which is the capital of Italy?"
    print(f"🗣️ User Query: '{query1}'")

    await run_agent_query(capital_agent, query1, capital_session, my_user_id)

    # Turn 2!
    query2 = "Which is the capital on the nation that was on the north of the previous one?"
    print(f"🗣️ User Query: '{query2}'")

    await run_agent_query(capital_agent, query2, capital_session, my_user_id)

    # Turn 3!
    query3 = "Which is the capital on the nation that was on the north of the previous one?"
    print(f"🗣️ User Query: '{query3}'")

    await run_agent_query(capital_agent, query3, capital_session, my_user_id)

await ask_for_nation_capital()

### 5) Using Agent without a memory

In [ ]:
async def ask_for_nation_capital():

    # Turn 1!
    query1 = "Which is the capital of Italy?"
    capital_session_one = await session_service.create_session(
        app_name=capital_agent.name,
        user_id=my_user_id
    )
    print(f"🗣️ User Query: '{query1}'")

    await run_agent_query(capital_agent, query1, capital_session_one, my_user_id)

    # Turn 2!
    query2 = "Which is the capital on the nation that was on the north of the previous one?"
    capital_session_two = await session_service.create_session(
        app_name=capital_agent.name,
        user_id=my_user_id
    )
    print(f"🗣️ User Query: '{query2}'")

    await run_agent_query(capital_agent, query2, capital_session_two, my_user_id)


await ask_for_nation_capital()

### 6) Use adk tool in a shell environment

In [ ]:
%%bash

rm -rf my-first-agent

adk create my-first-agent --model gemini-2.0-flash --api_key $GOOGLE_API_KEY

cat > my-first-agent/agent.py << EOF
from google.adk.agents import Agent


root_agent = Agent(
     name="capital_agent",
     model="gemini-2.0-flash",
     description="Answers user questions about the capital city of a given country.",
     instruction="""
     You are an agent that provides the capital city of a country.
     """
)
EOF


In [ ]:
!adk run my-first-agent